# GENERADOR DE GLOSAS EN LSE A PARTIR DE TEXTO

**Celia Botella López**

## 0. PREREQUISITOS

In [ ]:
import pandas as pd
import numpy as np
import re
import math
!pip install stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


## 1. FUNCIÓN DE GENERACIÓN DE GLOSAS

El generador que se implementa a continuación, transforma un texto escrito en español a su glosa en LSE. La función, denominada _`generatorText2Gloss()`_, recibe por parámetros:
* `input_text`: Secuencia de caracteres que forma el texto para el que se va a genera la glosa.
* `path_rules`: Ruta del fichero que contiene las reglas de transformación correspondientes a la LSE. Estas son las que se aplican a las frases para obtener la glosa.
* `path_corpus`: Ruta del fichero en el que se va a almacenar el corpus generado (pares de frases en español y glosa).
* `nlp`: Instancia de un pipeline de stanza.
* `max_len`: Longitud máxima que debe tener una frase para que sea transformada.

En primer lugar, se invoca a la función _`prepareText(text)`_, que modifica el formato del texto (string) a una lista de frases, donde cada frase es un listado de palabras, y cada palabra una instancia de la clase `Word`.

Seguidamente, se carga el conjunto de reglas de transformación almacenadas en el archivo (de tipo CSV) que recibe la función por parámetro. Para cada una de las frases que componen el texto, se aplican las reglas de transformación. Las reglas se aplican invocando a la fución _`applyRules(sentence, rules)`_, que espera por parámetro la frase y el listado de reglas. Esta función devuelve la misma frase con la información de las palabras actualizada: nueva posición que ocupa en la frase transformada y lema que representa su glosa.

Una vez que se obtiene la frase transformada, se invoca a la función _`glossSentece(sentence)`_, para formatear la frase (representada como una lista) a su correspondiente glosa en string.

Finalmente, se almacena en el fichero destino del corpus cada una de las frases del texto inicial junto a su resultado en glosas.

In [ ]:
def generatorText2Gloss(input_text, path_rules, path_corpus, nlp, max_len = 20):
    text = prepareText(input_text, nlp) ## Listado de frases

    rules = pd.read_csv(path_rules, encoding='utf-8') ## Lectura de reglas

    for sentence in text:
      if sentence != None:
        nwords = len(sentence)
        if nwords <= max_len:
          input_sentence = ' '.join([word.text for word in sentence]) ## Frase original

          ## Generar la glosa de la frase
  #        print("FRASE ORIGINAL: ", input_sentence)
          new_sentence = applyRules(sentence, rules, nlp) ## Aplicación de reglas
          gloss_sentence = glossSentece(new_sentence) ## Frase glosada
  #        print("\n\nRESULTADO GLOSA: ", gloss_sentence)

          ## Almacenamiento del resultado...
          insertCorpus(path_corpus, input_sentence, gloss_sentence)

  return ""

En definitiva, podemos diferenciar cuatro pasos principales: preparación de datos de entrada, aplicación de reglas, generación de glosa e inserción del resultado en el corpus. Para cada uno de ellos, se ha implementado una función.

### 1.1. PREPARACIÓN DE DATOS DE ENTRADA

La primera función _`prepareText(str_text, nlp)`_ recibe como entrada texto en formato string y devuelve el texto como un listado de frases, que a su vez son listas de palabras (instancias de la clase _`Word`_).

Para ello, se invoca al pipeline de la biblioteca *Stanza*, que separa el texto en frases por los signos de puntuación, tokeniza las frases en palabras, obtiene el lema de las palabras y analiza sintácticamente la frase, devolviendo información sobre la categoría gramatical cada una de las palabras que componen la oración.

Por cada frase del texto, se iteran sus palabras y se crea una instancia de la clase `Word` para cada una de ellas. La clase `Word` tiene los siguientes atributos: identificador (`id`), palabra textual (`text`), lema de la palabra (`lemma`), categoria gramatical universal (`upos`), categoría gramatical específica (`xpos`) y posición (`pos`) dentro de la oración a la que pertenece.

Las frases se define como una lista de instancias de la clase _`Word`_. Y el texto que devuelve la función será un listado de frases, o lo que es lo mismo, un listado de listas de palabras.

In [ ]:
class Word:
    def __init__(self, ident, text, lemma, upos, xpos, pos):
        self.id = ident
        self.text = text
        self.upper_text = str(text.upper())
        self.lemma = str(lemma.upper())
        self.upos = str(upos.lower()) if upos != None else ""
        self.xpos = str(xpos.lower()) if xpos != None else ""
        self.pos = pos


def prepareText(str_text, nlp):
    doc = nlp(str_text) ## Separa el texto en frases con palabras y sus caracteristicas
    text = []  ## Texto: listado de frases

    ## Iterar las frases extraidas
    for sent in doc.sentences:
        sentence = [] ## Frase: listado de palabras
        for word in sent.words:
            sentence.append(Word(word.id, word.text, word.lemma, word.upos, word.xpos, word.id-1))
        text.append(sentence) ## Insertar frase formateada en el texto

    return text

### 1.2. APLICACIÓN DE REGLAS

La función _`applyRule()`_ se encarga de aplicar las reglas a una frase. Recibe por parámetros:
* *sentence*: Listado de instancias de palabras.
* *rules*: Conjunto de reglas que se van a aplicar para transformar la frase.
* `nlp`: Instancia de un pipeline de stanza.

Las reglas se intentarán aplicar de una en una. Por tanto, comenzamos recorriéndolas. Cada regla tiene una estructura de entrada y una estructura de salida. Si se cumple la estructura de entrada, se transforma según la estructura de salida. Para trabajar mejor con las reglas, se modifica el formato en el que se representan las estructuras, de string a diccionario, con la función _`ruleAsDictionary(rule)`_ definifida.

Para encontrar qué palabras de la frase cumplen la estructura de entrada de una regla, invocamos a la función _`checkRule()`_. Dicha función devuelve las palabras que coinciden con el estructura de entrada de la regla. Puede ser que encuentre más de una solución (o coincidencia de estructura de entrada) en la frase, si la estructura se repite para varios conjuntos de palabras distintos.

Una vez encontrados los conjuntos de palabras que cumplen la entrada de una regla, hay cuatro opciones:
* Eliminar las palabras de entrada (input) que no se encuentran en la estructura de salida (output). Esto se hace modificando el valor de la posición de la palabra a -1.
* Insertar las palabras que no se encuentran en la estructura de entrada (input), pero sí están en la de salida (output). Las nuevas palabras son las que tienen el identificador a 0 en la estructura de salida. Al insertar la palabra, hay que tener en cuenta la posición en la que se encuentra dentro de la estructura de salida. Además, en estos casos, la descripción de la palabra en la estructura de salida será el lema que se utilice para el glosado de la palabra.
* Ordenar las palabras que se encuentran tanto en la estructura de entrada (input) como en la de salida (output), pero que en la salida están situadas en otra posición distinta a la de entrada. Para ello, se actualiza el valor del atributo de posición de cada palabra según se sitúen en la estructura de salida.
* Actualizar el lema de las palabras que se encuentran tanto en la estructura de entrada (input) como en la de salida (output), pero su descripción en la estructura de entrada es distinta a la que tiene en la de salida. En este caso, se modifica el lema de la palabra asociando la nueva descripción, según se indica en la estructura de salida.

Después de aplicar todas las reglas de transformación a la frase, se devuelve la misma frase de entrada pero con las posiciones de las palabras y sus lemas actualizados. Esta es la frase transformada que se utiliza para los pasos posteriores de generación de glosas e inserción en el corpus.


In [ ]:
def applyRules(sentence, rules, nlp):
    nwords = len(sentence) ## Numero de palabras en la frase
    pos = 1  ## Posición actual
    new_sentence = list(sentence) ## Frase transformada/actualizada
    exclusive_rules_applies = [] ## Reglas excluyentes aplicadas

    ## Iterar reglas de transformación
    for id_rule, row in rules.iterrows():

        ## Obtener la estructura de entrada y salida de la regla
        input_rule = ruleAsDictionary(row['input'])
        output_rule = ruleAsDictionary(row['output'])
        exclusive_rule = row['exclusive']


        ## Si la regla no es excluyente o no se ha aplicado otra que la excluya
        if exclusive_rule is None or not exclusive_rule in exclusive_rules_applies:
            ## Buscar las palabras de la frase que coinciden con la estructura de entrada
            new_sentence.sort(key=lambda x: x.pos)
            len_sentence = sum(word.pos>-1 for word in new_sentence)
            list_coincidences = checkRule(new_sentence, len_sentence, input_rule, 0, 1, [], [])

            if len(list_coincidences)>0:
                if not exclusive_rule is None:
                    exclusive_rules_applies.append(exclusive_rule) ## Insertar como aplicada

                for coincidence in list_coincidences:
#                    print("\n\nAPLICAR REGLA:")
#                    print("Input Rule: ", input_rule)
#                    print("Output Rule: ", output_rule)
#                    print("Coincidencia: ", coincidence)
                    coincidence_flatten = [j for sub in coincidence for j in sub]

                    ## Posicion del primer elemento que cumple con la regla
                    pos = next(word.pos for word in new_sentence if coincidence_flatten[0]==word.id)

                    ## Identificadores de las ultimas palabras que no cumplen con la regla
                    pos_last = next(word.pos for word in new_sentence if coincidence_flatten[-1]==word.id)
                    ids_last = [word.id for word in new_sentence if pos_last<word.pos]

                    ## Eliminar palabras del input rule que no estan en el output, poniendo posición a -1
                    ids_input_delete = list(iw for iw in list(input_rule) if iw not in list(output_rule))
                    for id_input_delete in ids_input_delete:
                        for id_delete in coincidence[id_input_delete-1]:
                          ## Busqueda de palabra con id a eliminar
                            for x in new_sentence:
                                if x.id == id_delete:
                                    x.pos = -1

                    ## Iterar las palabras en el output rule
                    for output_id in output_rule:
                        ## Añadir una nueva palabra, si el id en el output es 0 (no está en el input)
                        if output_id == 0:
                            nwords += 1
                            new_word_nlp = nlp(output_rule[output_id]).sentences[0].words[0]
                            new_word = Word(nwords, output_rule[output_id], output_rule[output_id], new_word_nlp.upos, new_word_nlp.xpos, pos)
                            new_sentence.append(new_word) ## Insertar palabra
                            pos += 1 ## Incrementar posición actual

                        ## Actualizar las palabras del output que estan en el input
                        elif output_id in list(input_rule) :

                            ## Descripción la palabra actual del input y output rule
                            input_word = input_rule[output_id]
                            output_word = output_rule[output_id]

                            for id_word in coincidence[output_id-1]:
                                ## Actualizar la posición de la palabra con la posición actual
                                current_word = next(x for x in new_sentence if x.id == id_word)
                                current_word.pos = pos
                                pos += 1 ## Incrementar posición actual

                                ## Si la descripcion del input y el output no coinciden
                                if(input_word != output_word):
                                    replace_word = output_word.replace(input_word, current_word.lemma.upper())
                                    current_word.lemma = replace_word   ## Modificar lema de la palabra

                    #Actualizar posiciones del resto de palabras de la frase
                    for word in new_sentence:
                        if word.id in ids_last:
                            word.pos = pos
                            pos += 1 ## Incrementar posición actual
                    new_sentence.sort(key=lambda x: x.pos)

                ## Imprimir la frase transformada tras aplicar una regla
#                print(glossSentece(new_sentence))

    return new_sentence


## Función que transforma el formato de una regla: de string a diccionario
## Las claves del diccionario sera la posición de las palabras (enteros)
## Los valores del diccionario sera la representación de las palabras (string)
def ruleAsDictionary(rule):
    dic_rule = {}
    ## Si la regla es nula, se devuelve un dict vacío
    if rule!=rule:
        return {}

    formatted_rule = rule.strip() ## Eliminar espacios del principio y fin
    formatted_rule = " ".join(formatted_rule.split()) ## Reemplazar multiples espacios por uno

    ## Separar el string por los delimitadores indicados
    list_rule = formatted_rule.split(" ")
    for elem in list_rule:
        ident, desc = elem.split("_", 1)
        dic_rule[int(ident)] = desc

    return dic_rule

La función _`checkRule()`_ es la que comprueba si una frase cumple una regla concreta. Se trata de una función de búsqueda recursiva. Recibe por parámetros:
* *sentence*: Frase como listado de palabras.
* *len_sentence*: Longitud de la frase.
* *rule*: Estructura de entrada de la regla que se va a comprobar si se cumple para la frase de entrada.
* *current_pos_word*: Posición de la palabra que se está comprobando actualmente.
* *current_pos_rule*: Posición de la parte de la regla que se está comprobando actualmente.
* *current_id*: Identificador de la palabra que se está comprobando actualmente.
* *current_solution*: Solución parcial encontrada hasta el momento (no definitiva). La solución es un listado de palabras.
* *solutions*: Listado con las soluciones definitivas encontradas. Cada solucion es un listado de palabras que cumplen con la regla.

Esta función, recorre las palabras de la frase de forma recursiva, comprobando si la posición y descripción de esa palabra coincide con la descripción que tiene la estructura de entrada de la regla (input rule) en esa posición. Al comprobar si una palabra cumple con la descripción, pueden ocurrir dos cosas:
* Que la descripción del input rule esté en minúscula. Se comprueba si el valor del campo descripción coincide con el del atributo upos o xpos de la palabra.
* Que la descripción del input rule esté en mayúscula. Se comprueba si el valor del campo descripción coincide con el del atributo texto o lema de la palabra.

Cuando en la regla aparezca una descripción con valor *, significará que cualquier palabra es válida, por lo que se insertan en la solución todas las palabras hasta encontrar otra que coincida con la siguiente descripción que aparece en la regla.

Las palabras que cumplan con la descripción, se insertan en el listado *current_solution* y se invoca a la función de nuevo para comprobar si la siguiente palabra coincide con la siguiente descripción de la regla. Una vez que se recorran todas las posiciones de la regla cumpliendose, la solución será definitiva y se almacena en el listado *solutions*. Si por el contrario, una palabra no cumple con la descripción de la regla, se descarta la solución parcial y se empieza a buscar una nueva solución.

Se siguen buscando soluciones hasta que se recorran todas las palabras de la frase.



In [ ]:
def checkRule(sentence, len_sentence, rule, current_pos_word=0, current_pos_rule=1, current_solution=[], solutions=[]):
    ## Si la longitud de la solución es la misma que la de la regla,
    ## se ha encontrado una solución completa
    if(len(current_solution) >= len(rule) and current_pos_rule > len(rule)):
        solutions.append(current_solution) ## Insertar la solución actual al listado de soluciones definitivas
        current_solution = [] ## Reiniciar la solucion actual
        current_pos_rule = 1 ## Reiniciar posicion

    ## Si la palabra actual es la última, FIN DE LA RECURSIVIDAD
    if(current_pos_word == len_sentence):
        return solutions ## Devolver las soluciones encontradas

    current_word = next((w for w in sentence if w.pos == current_pos_word),None)

    if len(current_solution) < (current_pos_rule):
        current_solution.append([])

    if(current_word == None):
        current_solution = []
        current_pos_rule = 1
    ## Comprobar si la palabra actual de la frase coincide con
    ## la descripción de la palabra actual de la regla
    elif(checkPartialRule(current_word, rule[current_pos_rule])):
        current_solution[current_pos_rule-1].append(current_word.id)
        current_pos_rule+=1
    elif rule[current_pos_rule] == '*':
        if(current_pos_rule+1 in rule.keys() and checkPartialRule(current_word, rule[current_pos_rule+1])):
            current_solution.append([current_word.id])
            current_pos_rule+=2
        else:
            current_solution[current_pos_rule-1].append(current_word.id)

        if(current_pos_word+1 == len_sentence):
            #Descartar la ultima parte de la regla si es *
            if(current_pos_rule == len(rule.keys()) and rule[current_pos_rule] == '*'):
                current_solution.append([])

            current_pos_rule+=1
    else:
        current_solution = []
        current_pos_rule = 1

    ## LLAMADA RECURSIVA: Comprobar si la siguiente palabra cumple
    return checkRule(sentence, len_sentence, rule, current_pos_word+1, current_pos_rule, current_solution, solutions)

In [ ]:
## Se comprueba si el valor de rule está en mayúscula y coincide con lemma
def isLemma(lemma, rule):
    return (rule.isupper() and (lemma == rule))

## Se comprueba si el valor de rule está en minúscula y coincide con upos o xpos
def isUPOS(upos, rule):
    return rule.islower() and (upos == rule)

def isXPOS(xpos, rule):
    if rule.islower() and (len(xpos) == len(rule)):
        arr_xpos = np.array(list(xpos))
        arr_rule = np.array(list(rule))

        zero_pos = np.where(arr_rule == '0') ## Posiciones con un 0
        arr_xpos[zero_pos] = '0' ## Sustituir posiciones por 0

        return np.array_equal(arr_xpos, arr_rule)

    return False

## Función que comprueba si una palabra coincide con la descripción de la regla
def checkPartialRule(word, rule):
    return (isLemma(word.lemma, rule) or isLemma(word.upper_text, rule) or isUPOS(word.upos, rule) or isXPOS(word.xpos, rule))

### 1.3. GENERACIÓN DE GLOSAS

La función _`glossSentece(sentence)`_, recibe una frase como lista de palabras (instancias de la clase `Word`). Se encarga de devolver un string con la glosa correspondiente a la frase de entrada. Para ello, se iteran las palabras de la frase y se concatena el lema de cada una en el orden que indica su atributo de posición (`pos`).

In [ ]:
def glossSentece(sentence):
    result = ""
    ## Iterar posiciones de la frase
    for pos in range(0, len(sentence)):
        current_word = next( (w for w in sentence if w.pos == pos), None) ## Busqueda de palabra en la posicion actual
        if current_word != None:
            result += current_word.lemma + " " ## Lema de la palabra en la posicion actual
    return result

### 1.4. INSERCIÓN EN EL CORPUS

La función _`insertCorpus()`_ inserta nuevos pares de frases (texto en español y su correspondiente glosa) en un fichero que contiene el corpus de datos. Recibe por parámetros:
* `path_corpus`. La ruta dónde se localiza el fichero con el corpus.
* `input_sentence`. Frase original en español (como string).
* `gloss_sentence.` Glosa generada en LSE (como string).

In [ ]:
def insertCorpus(path_corpus, input_sentence, gloss_sentence):
    df = pd.DataFrame({'sentence': [str(input_sentence)], 'gloss': [str(gloss_sentence)]})
    df.to_csv(path_corpus, mode='a', index=False, header=False)

## 2. PRUEBAS DE VALIDACIÓN DEL ALGORITMO

Una vez implementado el algoritmo de transformación a glosas, se debe establecer un método de validación que verifique que se aplican las reglas de manera correcta y devuelve el resultado en glosas esperado.

In [ ]:
## Si se está utilizando como entorno de trabajo Google Colab, hay que ejecutar esta celda
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/LSEGloss2SpanishText/

Para ello, se implementa la función `_test()_`, que recibe los siguientes parámetros:
* `path_sentences`. Ruta en la que se encuentra el corpus de pares de frases (español y glosa) con el que llevar a cabo las pruebas.
* `path_rules`: Ruta del fichero que contiene las reglas de transformación correspondientes a la LSE.
* `path_corpus`: Ruta del fichero en el que se va a almacenar el corpus generado (pares de frases en español y glosa).
* `nlp`: Instancia de pipeline de stanza.

Primero, se carga un conjunto de pares de frases en español y glosa, desde la ruta que se indica por parámetro. Seguidamente, a cada una de las frases cargadas en español, se le aplica el algoritmo de generación de glosa (se invoca a la función _`generatorText2Gloss()`_) y se comprueba si la glosa generada coincide con la glosa cargada. Si es así, el algoritmo habrá pasado la prueba.

In [ ]:
def test(path_sentences, path_rules, path_corpus, nlp):
    test_sentences = pd.read_csv(path_sentences, encoding='utf-8')

    total = len(test_sentences)
    passed = 0
    counter = 1
    for index, sentence in test_sentences.iterrows():
        gloss = generatorText2Gloss(sentence["Frase"], path_rules, path_corpus, nlp)
        if gloss.strip() == sentence["Glosa"].strip():
            print("\nTest ", counter, ".... passed")
            passed+=1
        else:
            print("Test ", counter, ".... no passed")
        counter+=1

        print("Frase original:", sentence["Frase"])
        print("Glosa generada:", gloss)

    print("\nTotal: ", passed, "/", total)

In [ ]:
test("Corpus/corpus-test.csv", "Gloss Generator/rules.csv", "Corpus/corpus-test-generated.csv", nlp)


Test  1 .... passed
Frase original: Pepe compró un coche a Pepa.
Glosa generada: PASADO PEPE-NP COCHE COMPRAR PEPA-NP 

Test  2 .... passed
Frase original: Llegaremos mañana al lugar previsto.
Glosa generada: MAÑANA NOSOTROS LLEGAR LUGAR PREVISTO 

Test  3 .... passed
Frase original: Cervantes escribió el Quijote
Glosa generada: PASADO CERVANTES-NP QUIJOTE-NP ESCRIBIR 

Test  4 .... passed
Frase original: Jesús lleva la camisa muy sucia.
Glosa generada: JESÚS-NP CAMISA SUCIO MUCHO LLEVAR 

Test  5 .... passed
Frase original: Rubén y Fernando no quieren ir al cine.
Glosa generada: RUBÉN-NP FERNANDO-NP IR QUERER NO CINE 

Test  6 .... passed
Frase original: Carlos le dio una carta certificada.
Glosa generada: PASADO CARLOS-NP CARTA CERTIFICADO DAR ÉL 

Test  7 .... passed
Frase original: La película es apta para menores
Glosa generada: PELÍCULA APTO MENOR 

Test  8 .... passed
Frase original: Informé a Gabriel de tu lesión.
Glosa generada: PASADO YO LESIÓN TU INFORMAR GABRIEL-NP 

Test 

## 3. GENERACIÓN DEL DATASET

In [ ]:
!pip install datasets
from datasets import load_dataset

## Carga del corpus de datos con frases en español.
dataset = load_dataset("PereLluis13/spanish_speech_text")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/2 [00:00<?, ?it/s]

Finalmente, se crea el corpus sintético final invocando a la función _`generatorText2Gloss()`_ y pasándole por parámetro las frases en español que queremos transformar a glosa. En este caso, utilizaremos las frases que contiene el corpus Spanish Speech Text.

In [ ]:
for text in dataset["clean"]["sentence"]:
    generatorText2Gloss(text, 'Gloss Generator/rules.csv', 'Corpus/corpus-spanish-gloss.csv', nlp)